In [70]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('/datasets/stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,stop_words,%d" % 1
            print "%s\t%d" % ("stopword", 1)
        print >> sys.stderr, "reporter:counter:Wiki stats,total_words,%d" % 1
        print "%s\t%d" % ("regularword", 1)

Overwriting mapper.py


In [71]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [72]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [73]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [74]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py, \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

hdfs dfs -cat ${OUT_DIR}/part-00001 | head

regularword	11937375
stopword	4588771


rm: `wordcount_result_1524684928816554': No such file or directory


		Stop words=4588771
		Total words=11937375


In [67]:
%%writefile get_stopword_percentage.py
import sys
import re

output_log = list(map(lambda x: x.strip(), sys.stdin.read().split()))

pattern_tot = 'total_words='
regexp_tot = re.compile(pattern_tot)

pattern_stop = 'stop_words='
regexp_stop = re.compile(pattern_stop)

total_words = [int(x.replace(pattern_tot, '')) for x in output_log if regexp_tot.search(x)][0]
stop_words = [int(x.replace(pattern_stop, '')) for x in output_log if regexp_stop.search(x)][0]

print(stop_words / float(total_words) * 100)

Overwriting get_stopword_percentage.py


In [75]:
%%bash

cat output.log | egrep "*_words=" | python get_stopword_percentage.py
# cat output.log >&2

38.44036900909957
